In [1]:
# from spdlog import ConsoleLogger, LogLevel
# name = 'Console Logger'
# logger = ConsoleLogger('Logger', False, True, True)
# def set_log_level(logger, level):
#     print("Setting Log level to %d" % level)
#     logger.set_level(level)

# set_log_level(logger, LogLevel.INFO)

import hashlib
from pyspark.sql import SparkSession
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import UserDefinedFunction,max,isnan, when, count, col, isnull,month, hour,year,minute,second,lower,to_timestamp,lit,udf,dayofweek, to_timestamp,trim
from pyspark.sql.types import TimestampType, DateType,DoubleType,FloatType,IntegerType,StringType
from pyspark.sql import DataFrame
from functools import reduce
import dateparser
import geopandas as gpd
import pandas as pd
from ftplib import FTP
from pyspark.ml.feature import StringIndexer, IndexToString
import os
from delta import *
import glob,sys
import pyarrow.parquet as pq
        
spark = SparkSession.builder.config('spark.executor.cores', '8')\
        .config('spark.executor.memory', '40g')\
        .config("spark.sql.session.timeZone", "UTC")\
        .config('spark.driver.memory', '20g')\
        .master("local[26]")\
        .appName("wego-daily")\
        .config('spark.driver.extraJavaOptions', '-Duser.timezone=UTC')\
        .config('spark.executor.extraJavaOptions', '-Duser.timezone=UTC')\
        .config("spark.sql.datetime.java8API.enabled", "true")\
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
        .config('spark.jars.packages', 'io.delta:delta-core_2.12:2.2.0')\
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")\
        .config('spark.databricks.delta.retentionDurationCheck.enabled',"false")\
        .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
        .getOrCreate()

pd.set_option('display.max_columns', None)


23/09/11 20:42:53 WARN Utils: Your hostname, Slade-173477 resolves to a loopback address: 127.0.1.1; using 10.2.219.10 instead (on interface eno1)
23/09/11 20:42:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/sgupta/anaconda3/envs/py39SAM/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sgupta/.ivy2/cache
The jars for the packages stored in: /home/sgupta/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e5a3609c-d8ca-4df5-beba-d3ab0db716b9;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.2.0 in central
	found io.delta#delta-storage;2.2.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 143ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.2.0 from central in [default]
	io.delta#delta-storage;2.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0

In [2]:
import pandas as pd
import numpy as np

In [5]:
DATA_DIR = '/home/sgupta/WORK/DATASETS/WeGo_Bus_data/STOPLEVEL_PROCESSED'
data_fp = f'{DATA_DIR}/final_stop_level_data_samir.parquet'

df = pd.read_parquet(f'{data_fp}')

In [7]:
df['apc_signup_name'] = np.where(((df.transit_date >= '2020-09-13') & (df.transit_date <= '2021-04-10')),'September 2020',
                                np.where(((df.transit_date >= '2021-04-11') & (df.transit_date <= '2021-10-02')),'April 2021',
                                    np.where(((df.transit_date >= '2021-10-03') & (df.transit_date <= '2022-04-02')),'October 2021',
                                        np.where(((df.transit_date >= '2022-04-03') & (df.transit_date <= '2022-10-01')),'April 2022',
                                            np.where(((df.transit_date >= '2022-10-02') & (df.transit_date <= '2023-04-01')),'October 2022',
                                                np.nan
                                            )
                                        )
                                    )
                                )
                              )

In [8]:
apc_data = df

In [9]:
apc_data = apc_data[(apc_data['route_direction_name'] == 'TO DOWNTOWN') | (apc_data['route_direction_name'] == 'FROM DOWNTOWN')]

In [ ]:
raw_apc = pd.read_parquet('../../DATASETS/WeGo_Bus_data/RAW/wego-daily.apc.parquet')

### merging with RAW APC and saving per route data

In [11]:
route_ids = apc_data.route_id.unique()

In [12]:
route_ids

array([23, 52, 50,  4, 55, 28,  3,  8, 17,  5,  7, 22, 18, 56, 29, 19, 14,
       34, 42,  6,  9, 41], dtype=int32)

In [26]:
from pathlib import Path
import pathlib

In [27]:
def merge_w_apc_raw(apc_df,raw_df,route_id):
    # Calculating the delay
    raw_df['delay'] = raw_df['arrival_time'] - raw_df['scheduled_time'] 
    raw_df['delay'] = raw_df['delay'].dt.total_seconds().div(60).astype(int,errors='ignore')

    raw_df = raw_df[['route_id','transit_date', 'route_direction_name', 'trip_id', 'block_abbr', 'pattern_num','stop_sequence','map_latitude','map_longitude','sched_hdwy','actual_hdwy','delay']]

    apc_df = apc_df.merge(raw_df,on=['route_id','transit_date', 'route_direction_name', 'trip_id', 'block_abbr', 'pattern_num','stop_sequence'],how='left')

    op_folder = f'./data/route_apc/{route_id}'
    op_file = f'data_route_{route_id}.parquet'

    pathlib.Path(op_folder).mkdir(parents=True, exist_ok=True) 
    
    apc_df.to_parquet(f'{op_folder}/{op_file}')

In [28]:
from tqdm import tqdm
for route_id in tqdm(route_ids):
    # display(apc_data[apc_data['route_id'] == route_id])
    # display(raw_apc[raw_apc['route_id'] == route_id])
    merge_w_apc_raw(apc_data[apc_data['route_id'] == route_id],raw_apc[raw_apc['route_id'] == route_id],route_id)
    # break
    # merge_w_apc_raw(apc_df,raw_df,route_id)

  0%|          | 0/22 [00:00<?, ?it/s]/tmp/ipykernel_3597326/3296439113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['delay'] = raw_df['arrival_time'] - raw_df['scheduled_time']
/tmp/ipykernel_3597326/3296439113.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df['delay'] = raw_df['delay'].dt.total_seconds().div(60).astype(int,errors='ignore')
  5%|▍         | 1/22 [00:10<03:33, 10.17s/it]/tmp/ipykernel_3597326/3296439113.py:3: SettingWithCopyWarning: 
A value is trying to be set on a c

### Sanity Checks

In [13]:
for route_id in route_ids:
    print("Route_id: ",route_id)

    op_folder = f'./data/route_apc/{route_id}'
    op_file = f'data_route_{route_id}.parquet'
    temp = pd.read_parquet(f'{op_folder}/{op_file}')

    if((len(temp[temp.map_latitude.isna()]) != 0) | (len(temp[temp.map_longitude.isna()]) != 0)):
        display(len(temp[temp.map_latitude.isna()]))
        display(len(temp[temp.map_longitude.isna()]))
        display(len(temp[temp.delay.isna()]))

Route_id:  23
Route_id:  52
Route_id:  50
Route_id:  4
Route_id:  55
Route_id:  28
Route_id:  3
Route_id:  8
Route_id:  17
Route_id:  5
Route_id:  7
Route_id:  22
Route_id:  18
Route_id:  56
Route_id:  29
Route_id:  19
Route_id:  14
Route_id:  34
Route_id:  42
Route_id:  6
Route_id:  9
Route_id:  41
